In [1]:
import pandas as pd
import numpy as np
import math
import datetime
import bandit.main as bd

In [2]:
def get_frame(filename):
    df = pd.read_excel(io=filename)
    # df_combined = pd.merge(df,df_lookup,on='Difficulty')
    # df_combined['Duration'] = df_combined['Time'].apply(lambda x: pd.Timedelta(str(x))).astype('timedelta64[s]')
    # df_combined['MatRate'] = df_combined['Mats'] / df_combined['Duration']
    # df_combined['DiffClass'] = df_combined['Class'] + '-' + df_combined['Difficulty']
    return df

In [3]:
def get_ucb(row):
    div1 = 2 * math.log(row['total']) / row['count']
    ucb = math.sqrt(div1)
    return ucb + row['median']

In [4]:
def get_ucb2(row):
    div1 = 3 * math.log(row['total']) / row['count']
    min1 = min(0.25, row['MAD'] + div1)
    div2 = math.log(row['total']) / row['count']
    ucb = math.sqrt(div2 * min1)
    return ucb + row['median']

In [5]:
def save_excel(df,col):
    dfgb = df.groupby([col])
    dfgb = dfgb['Quality'].agg(['count','median', bd.mad_function]).rename(columns={'mad_function': 'MAD'}).reset_index()
    return dfgb

In [6]:
df = get_frame('ProjectTime.xlsx')

In [7]:
# reduce it
df = bd.reduce(df,'Project')
df['Duration'] = df['Duration'].apply(lambda x: pd.Timedelta(str(x))).astype('timedelta64[s]')
df_quality = bd.get_quality(df['Duration'])
df = df.join(df_quality)

28 0.9299999999999999 26


In [8]:
df.tail(10)

,Project,ProjeDate,Duration,Quality
30,ZeroToDeepLearn,2018-07-22,2748.0,0.839800
31,GoogleCrashCourse,2018-07-24,2520.0,0.768461
32,ZeroToDeepLearn,2018-07-25,2410.0,0.734043
33,GoogleCrashCourse,2018-07-29,2880.0,0.881101
34,SpotifyPlayGen,2018-07-31,2153.0,0.653630
35,SpotifyPlayGen,2018-08-02,2584.0,0.788486
36,SpotifyPlayGen,2018-08-04,3260.0,1.000000
37,SpotifyPlayGen,2018-08-05,1932.0,0.584481
38,GoogleCrashCourse,2018-08-06,1743.0,0.525344
39,KaggleLearn,2018-08-10,1287.0,0.382666


In [9]:
df.sort_values('Quality', ascending=False).head(10)

,Project,ProjeDate,Duration,Quality
36,SpotifyPlayGen,2018-08-04,3260.0,1.000000
33,GoogleCrashCourse,2018-07-29,2880.0,0.881101
30,ZeroToDeepLearn,2018-07-22,2748.0,0.839800
15,SpecCurator,2018-06-21,2719.0,0.830726
29,ZeroToDeepLearn,2018-07-20,2620.0,0.799750
35,SpotifyPlayGen,2018-08-02,2584.0,0.788486
31,GoogleCrashCourse,2018-07-24,2520.0,0.768461
32,ZeroToDeepLearn,2018-07-25,2410.0,0.734043
2,CUDAproject,2018-05-05,2289.0,0.696183
3,SpecCurator,2018-06-08,2198.0,0.667710


In [10]:
dfgb = save_excel(df,'Project')

In [11]:
df = df.merge(dfgb)
dfgb['total'] = dfgb['count'].sum()
dfgb['donext'] = dfgb.apply(get_ucb2, axis=1)
dfgb.sort_values('donext', ascending=False)

,Project,count,median,MAD,total,donext
8,SpotifyPlayGen,4,0.721058,0.102003,38,1.197869
11,ZeroToDeepLearn,5,0.734043,0.099812,38,1.160516
2,FindYourJoy,3,0.580726,0.037234,38,1.131300
0,CUDAproject,4,0.651909,0.029568,38,1.128720
3,GoogleCrashCourse,4,0.646902,0.177879,38,1.123714
10,VueClass,2,0.448842,0.011108,38,1.123155
9,TrollSniffer,3,0.561952,0.016896,38,1.112527
1,DjangoClass,2,0.418335,0.056633,38,1.092649
7,SpecCurator,4,0.571809,0.108886,38,1.048620
4,KaggleLearn,3,0.429912,0.047247,38,0.980487
